# Take a close look at extreme p-values

In [1]:
library(data.table)

In [2]:
bigdata <- fread("16a9par-OUT_stage2_MWAS_scz.csv")

In [3]:
colnames(bigdata)

[1] "z"          "p"          "n"          "chr"        "pos"       
[6] "population" "region"     "stats"      "scaff"

In [5]:
smallerdata <- bigdata[which(bigdata$p < 10^-200), ]

In [6]:
#smallerdata <- bigdata[which(bigdata$p < 10^-200), ]

dim(smallerdata)

fwrite(smallerdata, "22_intermediate_extreme_stage2_MWAS_scz.csv")

[1] 4919    9

In [ ]:
smallerdata <- fread("22_intermediate_extreme_stage2_MWAS_scz.csv")

### What are the stage 1 weights for SNPs contributing to this CpG site?

In [7]:
#weights <- readRDS("/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_a2_caud/libd_chr1-chr1_AA-libd_chr1-chr1_AA-908982-928981-dynamic-1corestotal-allcorepera-20240415-134429.rds")

In [8]:
weights <- readRDS("/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/output_EXPANSE_a2_caud/libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-20240416-172011.rds")

In [9]:
smallerdata <- smallerdata[which(smallerdata$population == "all" & smallerdata$region == "caud"), ]

In [10]:
library(data.table)

#smallerdata_sorted <- smallerdata[order(smallerdata$pos), ]
#duplicated_pos <- smallerdata_sorted$pos[duplicated(smallerdata_sorted$pos)]
#smallerdata_sorted[smallerdata_sorted$pos %in% duplicated_pos, ]

smallerdata[, timestamp := sub(".*-(\\d{8}-\\d{6})$", "\\1", scaff)]
smallerdata <- smallerdata[order(pos, -timestamp)]

smallerdata <- smallerdata[, .SD[1], by = pos]

In [11]:
methylation_positions <- vector("list", length(weights@models))
snp_weights_list_of_lists <- vector("list", length(weights@models))
desired_methylation_positions <- vector("list", sum(smallerdata$pos %in% sapply(weights@models, function(x) x@methylationPosition)))
desired_snp_weights_list_of_lists <- vector("list", sum(smallerdata$pos %in% sapply(weights@models, function(x) x@methylationPosition)))
desired_snp_pvals <- vector("list", sum(smallerdata$pos %in% sapply(weights@models, function(x) x@methylationPosition)))

In [12]:
pos_index <- 1

for(i in seq_along(weights@models)){
    methylation_positions[[i]] <- weights@models[[i]]@methylationPosition
    snp_weights_list_of_lists[[i]] <- weights@models[[i]]@snpWeights

    if(weights@models[[i]]@methylationPosition %in% smallerdata$pos){
        desired_methylation_positions[[pos_index]] <- weights@models[[i]]@methylationPosition
        if(is.null(weights@models[[i]]@snpWeights)) stop("problem")
        desired_snp_weights_list_of_lists[[pos_index]] <- weights@models[[i]]@snpWeights
        desired_snp_pvals[[pos_index]] <- smallerdata$p[which(smallerdata$pos == weights@models[[i]]@methylationPosition)]
        pos_index <- pos_index + 1
    }
}

#methylation_positions <- unlist(methylation_positions)
#desired_methylation_positions <- unlist(desired_methylation_positions)
#desired_snp_pvals <- unlist(desired_snp_pvals)

In [16]:
smallerdata_sub <- smallerdata[which(smallerdata$pos >= 73274305-20000 & smallerdata$pos <= 73419830 + 20000)]

In [17]:
smallerdata_sub

pos,z,p,n,chr,population,region,stats,scaff,timestamp
<int>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
73274305,51.28433,0.000000e+00,6,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818
73274312,51.03427,0.000000e+00,6,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818
73292330,39.13517,0.000000e+00,9,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818
73307769,-31.18148,1.899807e-213,4,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818
73308571,-55.73635,0.000000e+00,22,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818
73419188,-36.40970,2.990203e-290,23,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818
73419830,-34.07063,2.009205e-254,20,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818
73420076,-31.35857,7.433658e-216,20,1,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr1-chr1_all-libd_chr1-chr1_all-908982-928981-dynamic-1corestotal-allcorepera-caud-20240510-145818,20240510-145818


In [19]:
smallerdata_sub$pos

[1] 73274305 73274312 73292330 73307769 73308571 73419188 73419830 73420076

In [15]:
smallerdata

pos,z,p,n,chr,population,region,stats,scaff,timestamp
<int>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
651107,-31.10433,2.104787e-212,46,2,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr2-chr2_all-libd_chr2-chr2_all-1-20000-dynamic-1corestotal-allcorepera-caud-20240511-065518,20240511-065518
1988129,-33.45159,2.440211e-245,2,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351
1988133,-33.32656,1.592389e-243,2,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351
1988141,-31.93231,9.512265e-224,2,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351
2006943,-30.46161,8.407191e-204,4,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351
2007013,-35.11031,4.692392e-270,3,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351
2007144,-35.97829,1.828241e-283,5,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351
2007161,-33.88309,1.182130e-251,4,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351
2007164,-33.87557,1.525460e-251,4,7,all,caud,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr7-chr7_all-libd_chr7-chr7_all-40001-60000-dynamic-1corestotal-allcorepera-20240422-130351,20240422-130351


In [13]:
desired_methylation_positions

[[1]]
[1] 73244297

[[2]]
[1] 73244437

[[3]]
[1] 73244564

[[4]]
[1] 73274305

[[5]]
[1] 73274312

[[6]]
[1] 73292330

[[7]]
[1] 73307769

[[8]]
[1] 73308571

[[9]]
[1] 73419188

[[10]]
[1] 73419830

[[11]]
[1] 73420076

In [14]:
desired_snp_pvals

[[1]]
[1] 2.117459e-222

[[2]]
[1] 6.69425e-217

[[3]]
[1] 2.284288e-204

[[4]]
[1] 0

[[5]]
[1] 0

[[6]]
[1] 0

[[7]]
[1] 1.899807e-213

[[8]]
[1] 0

[[9]]
[1] 2.990203e-290

[[10]]
[1] 2.009205e-254

[[11]]
[1] 7.433658e-216

In [ ]:
# Combine all elements into a single data.table
df <- data.table(
  methylation_position = rep(desired_methylation_positions, lengths(desired_snp_weights_list_of_lists)),
  pval_mwas_stage2 = rep(desired_snp_pvals, lengths(desired_snp_weights_list_of_lists)),
  contributing_snp_name = unlist(lapply(desired_snp_weights_list_of_lists, function(x) sapply(strsplit(names(x), ":"), function(y) paste(y[1:4], collapse=":")))),
  contributing_snp_weight_mwas_stage1_prediction = unlist(desired_snp_weights_list_of_lists)
)


In [ ]:
library(stringr)

In [ ]:
df$contributing_snp_pos <- as.numeric(str_split_fixed(df$contributing_snp_name, ":", 3)[, 2])

In [ ]:
head(df)

### What are the summary stat weights and SE for the same SNPs?

In [ ]:
library(data.table)

In [ ]:
ss_path <- "/home/naglemi/mwas/gwas/gwas_stat_scz"

In [ ]:
snp.gwas2 <- fread(ss_path, skip = 1, header = FALSE)
colnames(snp.gwas2) <- strsplit(readLines(ss_path, n = 1), "\t")[[1]]

In [ ]:
snp.gwas2 <- snp.gwas2[which(snp.gwas2$CHR == 1), ]

In [ ]:
colnames(snp.gwas2) <- paste0("ss_", colnames(snp.gwas2))

In [ ]:
colnames(snp.gwas2)[which(colnames(snp.gwas2)  == "ss_BP")] <- "contributing_snp_pos"

In [ ]:
merged <- merge(df, snp.gwas2, by = "contributing_snp_pos")

In [ ]:
# make sure we have same allele orientation in both. TRUE if so.
merged$contributing_snp_name_verified <- paste0("chr1:",
                                                merged$contributing_snp_pos,
                                                ":",
                                                merged$ss_A1,
                                                ":",
                                                merged$ss_A2)

all(merged$contributing_snp_name_verified == merged$contributing_snp_name)

In [ ]:
# should all be unique
contributing_snp_pos_methylation_position_combo <- paste0(merged$contributing_snp_pos,
                                                          "_",
                                                          merged$methylation_position)
length(contributing_snp_pos_methylation_position_combo) == length(unique(contributing_snp_pos_methylation_position_combo))

In [ ]:
view_cols <- c("contributing_snp_pos", "contributing_snp_name", "contributing_snp_name_verified", "ss_CHR", "ss_SNP", "ss_A1", "ss_A2")
#merged[, ..view_cols]


### Recompute the stage 2 result

### First we need the SNP data

#### Load all SNP data

In [ ]:
library(CpGWAS)

In [ ]:
paths <- list(pvar_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/ref_EUR_chr1.pvar",
              pgen_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/ref_EUR_chr1.pgen",
              psam_path = "/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/ref_EUR_chr1.psam")

my_SNPs <- loadSNPData(paths$pvar_path, paths$pgen_path, paths$psam_path)

Now let's practice subsetting the SNP data to get the ones we want and need for a given methylation site

#### Get the SNP data for specific SNPs contributing to methylation site

##### What are those SNPs?

In [ ]:
library(pgenlibr)

In [ ]:
i <- 1

In [ ]:
this_desired_methylation_position <- unlist(desired_methylation_positions[i])

In [ ]:
this_desired_methylation_position

In [ ]:
typeof(merged$methylation_position)
typeof(this_desired_methylation_position)

In [ ]:
merged_this_methylation_position <- merged[methylation_position == this_desired_methylation_position, ]

In [ ]:
snp_indices_of_interest <- which(my_SNPs$pvar_dt$POS %in% merged_this_methylation_position$contributing_snp_pos)

##### Extract and format the SNPs

In [ ]:
G <- pgenlibr::ReadList(my_SNPs$pgen,
                        variant_subset = snp_indices_of_interest)

In [ ]:
dim(G)

In [ ]:
colnames(G) <- my_SNPs$pvar_dt$ID[snp_indices_of_interest]
rownames(G) <- my_SNPs$psam$IID

In [ ]:
dim(G)

In [ ]:
G

### stage 2 MWAS

In [ ]:
merged_this_methylation_position$logOR <- log(merged_this_methylation_position$ss_OR)
# merged_this_methylation_position$SElogOR <- merged_this_methylation_position$ss_SE / merged_this_methylation_position$ss_OR

# wrong way to compute it - no actually right
merged_this_methylation_position$z <- merged_this_methylation_position$logOR / merged_this_methylation_position$ss_SE

# The (right?) way to compute it? - no wrong
# merged_this_methylation_position$z <- merged_this_methylation_position$logOR / merged_this_methylation_position$SElogOR

In [ ]:
merged_this_methylation_position$ss_SE

In [ ]:
merged_this_methylation_position

In [ ]:
merged_this_methylation_position$ss_P

In [ ]:
mwas <- function(z, w, G){   
  if(length(w) > 1){
    #recover()
    # z-scores for effect of SNPs on external phenotype
    #. are weighted according to weights for effect of SNPs on methylation
    z <- z %*% w
    print(paste0("Weighted z-score is: ", z))
    # compute correlation matrix of SNP matrix, which captures LD structure
    print("We are looking at head of dosage data in genotype matrix G:")
    print(head(G))
    z.cor <- cor(G)
    print(paste0("z.cor is ", z.cor))
    # add small value to diagonal to avoid singular matrix
    #  which may otherwise happen if two SNPs in perfect LD
    z.cor <- z.cor + diag(dim(z.cor)[1])*0.1 
    # variance of correlated variables is weighted sum 
    # multiplying w by corr matrix once gives a vector representing
    #. the variance of each individual SNP and the extent to which they are
    #. influenced by other SNPs. Multiplying again by w sums up pairwise contributions
    #. and reflects total variance of weighted sum.
    #. the first w is automatically transposed by R so we don't have to write t(w)
    se <- sqrt(w %*%  z.cor %*%  w)
    print(paste0("Standard error is: ", se))
    z <- z/se
    p <- pnorm(abs(z), lower.tail=F)*2
    return(c(z=z, p=p, n=length(w)))
  } else {
    p <- pnorm(abs(z), lower.tail=F)*2
    return(c(z=z, p=p, n=1))
  }
}

In [ ]:
mwas(z = merged_this_methylation_position$z,
     w = merged_this_methylation_position$contributing_snp_weight_mwas_stage1_prediction,
     G = G)

In [ ]:
mwas(z = merged_this_methylation_position$z,
     w = merged_this_methylation_position$contributing_snp_weight_mwas_stage1_prediction,
     G = G)

##### An earlier attempt where we used ss instead of the merged df that contains it and other stuff too

In [ ]:
ss_snps_for_this_cpg <- snp.gwas2[which(snp.gwas2$contributing_snp_pos %in% merged_this_methylation_position$contributing_snp_pos), ]

In [ ]:
ss_snps_for_this_cpg <- ss_snps_for_this_cpg[order(ss_snps_for_this_cpg$contributing_snp_pos), ]

In [ ]:
ss_snps_for_this_cpg

In [ ]:
ss_snps_for_this_cpg$logOR <- log(ss_snps_for_this_cpg$ss_OR)
ss_snps_for_this_cpg$SElogOR <- ss_snps_for_this_cpg$ss_SE / ss_snps_for_this_cpg$ss_OR

In [ ]:
# The (right?) way to compute it?
ss_snps_for_this_cpg$z <- ss_snps_for_this_cpg$logOR / ss_snps_for_this_cpg$SElogOR

In [ ]:
ss_snps_for_this_cpg

In [ ]:
dim(merged)

In [ ]:
colnames(merged)

In [ ]:
head(merged)

In [ ]:
mwas(z = ss_snps_for_this_cpg$z,
     w = ss_snps_for_this_cpg$)

In [ ]:
mwas <- function(z, w, G){   
  if(length(w) > 1){
    #recover()
    # z-scores for effect of SNPs on external phenotype
    #. are weighted according to weights for effect of SNPs on methylation
    z <- z %*% w
    # compute correlation matrix of SNP matrix, which captures LD structure
    z.cor <- cor(G)
    # add small value to diagonal to avoid singular matrix
    #  which may otherwise happen if two SNPs in perfect LD
    z.cor <- z.cor + diag(dim(z.cor)[1])*0.1 
    # variance of correlated variables is weighted sum 
    # multiplying w by corr matrix once gives a vector representing
    #. the variance of each individual SNP and the extent to which they are
    #. influenced by other SNPs. Multiplying again by w sums up pairwise contributions
    #. and reflects total variance of weighted sum.
    #. the first w is automatically transposed by R so we don't have to write t(w)
    se <- sqrt(w %*%  z.cor %*%  w)
    z <- z/se
    p <- pnorm(abs(z), lower.tail=F)*2
    return(c(z=z, p=p, n=length(w)))
  } else {
    p <- pnorm(abs(z), lower.tail=F)*2
    return(c(z=z, p=p, n=1))
  }
}

In [ ]:
mwas()

In [ ]:
getwd()

In [ ]:
# List files recursively in the current working directory
files <- list.files(path = "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/", pattern = "", full.names = TRUE, recursive = TRUE)

# Filter files containing both 'chr1-chr1' and '_results.rds'
files_to_rename <- grep("chr1-chr1.*_results\\.rds", files, value = TRUE)

In [ ]:
# Rename files using a loop
for (file in files_to_rename) {
  new_name <- gsub("_results\\.rds", "_oldresults.rds", file)
  #print(file)
  #print(new_name)
  #print("next")
  file.rename(file, new_name)
}